# SQL-with-PySpark

Load `chinook` data into PostgreSQL - `pg-data`

- [Chinook_PostgreSql.sql](https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_PostgreSql.sql)

In [1]:
# Making sure to link pyspark to the right Spark folder with findspark
import findspark
import time
from pathlib import Path
from functools import wraps
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, sum, year, row_number, desc
findspark.init('/opt/spark')

### Sugar syntax purpose

In [2]:
class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

## Downloading dependencies

In [3]:
! apt-get install -y curl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.24).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [4]:
! curl -O https://jdbc.postgresql.org/download/postgresql-42.6.0.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1056k  100 1056k    0     0   467k      0  0:00:02  0:00:02 --:--:--  467k


In [5]:
spark = SparkSession.builder.appName(
    "SQL-with-PySpark"
).config(
    "spark.jars",
    f"{str(Path.cwd())}/postgresql-42.6.0.jar"
).config(
    "spark.driver.extraClassPath",
    f"{str(Path.cwd())}/postgresql-42.6.0.jar"
).getOrCreate()

## Loading Tables from Relational Database

### Tables:

- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track

In [6]:
tables = [
    "Album",
    "Artist",
    "Customer",
    "Employee",
    "Genre",
    "Invoice",
    "InvoiceLine",
    "MediaType",
    "Playlist",
    "PlaylistTrack",
    "Track",
]

In [7]:
tmp = {}
for table in tables:
    tmp[table.lower()] = spark.read.format(
        "jdbc"
    ).options(
        url="jdbc:postgresql://pg-data:5433/postgres",
        dbtable=f"chinook.\"{table}\"",
        driver="org.postgresql.Driver",
        user="postgres",
        password="postgres"
    ).load()
df = AttributeDict(tmp)

In [8]:
df.album.printSchema()

root
 |-- AlbumId: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- ArtistId: integer (nullable = true)



In [9]:
df.artist.printSchema()

root
 |-- ArtistId: integer (nullable = true)
 |-- Name: string (nullable = true)



In [10]:
df.invoice.printSchema()

root
 |-- InvoiceId: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- BillingAddress: string (nullable = true)
 |-- BillingCity: string (nullable = true)
 |-- BillingState: string (nullable = true)
 |-- BillingCountry: string (nullable = true)
 |-- BillingPostalCode: string (nullable = true)
 |-- Total: decimal(10,2) (nullable = true)



In [11]:
df.invoiceline.printSchema()

root
 |-- InvoiceLineId: integer (nullable = true)
 |-- InvoiceId: integer (nullable = true)
 |-- TrackId: integer (nullable = true)
 |-- UnitPrice: decimal(10,2) (nullable = true)
 |-- Quantity: integer (nullable = true)



In [12]:
df.customer.printSchema()

root
 |-- CustomerId: integer (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Fax: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- SupportRepId: integer (nullable = true)



## Start querying

### join example

In [13]:
df.album.join(
    df.artist, df.album.ArtistId == df.artist.ArtistId, "inner"
).select(
    df.album.AlbumId, df.album.Title, df.artist.Name.alias("ArtistName")
).show()

+-------+--------------------+--------------------+
|AlbumId|               Title|          ArtistName|
+-------+--------------------+--------------------+
|    228|    Heroes, Season 1|              Heroes|
|    308|Tchaikovsky: 1812...|Antal Dor�ti & Lo...|
|     83|My Way: The Best ...|       Frank Sinatra|
|    209|       Live [Disc 1]|    The Black Crowes|
|    210|       Live [Disc 2]|    The Black Crowes|
|    319|Armada: Music fro...|            Fretwork|
|     38|  Heart of the Night|          Spyro Gyra|
|    204|       Morning Dance|          Spyro Gyra|
|    325|Bartok: Violin & ...|      Yehudi Menuhin|
|    205|             In Step|Stevie Ray Vaugha...|
|     67|Vault: Def Leppar...|         Def Leppard|
|    161|          Demorou...|     M�nica Marianno|
|    248|    Ao Vivo [IMPORT]|      Zeca Pagodinho|
|    277|Bach: Goldberg Va...|      Wilhelm Kempff|
|    142|Lulu Santos - RCA...|         Lulu Santos|
|    143|Lulu Santos - RCA...|         Lulu Santos|
|    175|Wal

### aggregate example

In [14]:
dff = df.invoice.join(
    df.customer, df.invoice.CustomerId == df.customer.CustomerId, "inner"
).groupBy(
    year(df.invoice.InvoiceDate).alias("Year"), df.customer.CustomerId
).agg(
    sum(df.invoice.Total).alias("Total")
)

### window example

In [15]:
window = Window.partitionBy(
    "Year"
).orderBy(
    desc("Total")
)

In [16]:
dff.withColumn(
    "idx", row_number().over(window)
).filter(
    col("idx").isin(1, 2, 3)
).show()

+----+----------+-----+---+
|Year|CustomerId|Total|idx|
+----+----------+-----+---+
|2013|         6|27.84|  1|
|2013|        35|24.75|  2|
|2013|        56|24.75|  3|
|2012|        26|25.84|  1|
|2012|        34|24.77|  2|
|2012|        13|24.75|  3|
|2009|        40|24.75|  1|
|2009|        19|24.75|  2|
|2009|         2|24.75|  3|
|2010|        45|32.75|  1|
|2010|         7|27.77|  2|
|2010|         3|26.75|  3|
|2011|        46|32.75|  1|
|2011|         8|24.75|  2|
|2011|        29|24.75|  3|
+----+----------+-----+---+

